In [1]:
import pandas as pd
import tensorflow as tf

2022-07-01 09:50:22.482482: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-01 09:50:22.482503: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
df = pd.read_csv('reviews.csv') 
df.head()

,ReviewId,RecipeId,AuthorId,AuthorName,Rating,Review,DateSubmitted,DateModified
0,2,992,2008,gayg msft,5,better than any you can get at a restaurant!,2000-01-25T21:44:00Z,2000-01-25T21:44:00Z
1,7,4384,1634,Bill Hilbrich,4,"I cut back on the mayo, and made up the differ...",2001-10-17T16:49:59Z,2001-10-17T16:49:59Z
2,9,4523,2046,Gay Gilmore ckpt,2,i think i did something wrong because i could ...,2000-02-25T09:00:00Z,2000-02-25T09:00:00Z
3,13,7435,1773,Malarkey Test,5,easily the best i have ever had. juicy flavor...,2000-03-13T21:15:00Z,2000-03-13T21:15:00Z
4,14,44,2085,Tony Small,5,An excellent dish.,2000-03-28T12:51:00Z,2000-03-28T12:51:00Z


In [3]:
df.isnull().sum()

ReviewId           0
RecipeId           0
AuthorId           0
AuthorName         0
Rating             0
Review           214
DateSubmitted      0
DateModified       0
dtype: int64

In [4]:
df = df.dropna(subset=['Review'])
df.isnull().sum()

ReviewId         0
RecipeId         0
AuthorId         0
AuthorName       0
Rating           0
Review           0
DateSubmitted    0
DateModified     0
dtype: int64

In [5]:
df['cleaned_reviews'] = df['Review'].str.lower()

In [6]:
df['cleaned_reviews'].head()

0         better than any you can get at a restaurant!
1    i cut back on the mayo, and made up the differ...
2    i think i did something wrong because i could ...
3    easily the best i have ever had.  juicy flavor...
4                                   an excellent dish.
Name: cleaned_reviews, dtype: object

In [7]:
df['cleaned_reviews'] = df['cleaned_reviews'].str.replace('[^\w\s]','') 
df['cleaned_reviews'].head() 


/tmp/ipykernel_1286053/3349767945.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['cleaned_reviews'] = df['cleaned_reviews'].str.replace('[^\w\s]','')


0          better than any you can get at a restaurant
1    i cut back on the mayo and made up the differe...
2    i think i did something wrong because i could ...
3    easily the best i have ever had  juicy flavorf...
4                                    an excellent dish
Name: cleaned_reviews, dtype: object

In [8]:
import nltk
from nltk.corpus import stopwords 

In [9]:
# nltk.download('stopwords')

In [10]:
stop_words = stopwords.words('english')

In [11]:
def remove_sw(text):
    filtered_words = []
    for x in text: 
        if x not in stop_words:
            filtered_words.append(x)
    
    return filtered_words 

In [12]:
df['cleaned_reviews'] = df['cleaned_reviews'].apply(lambda x: x.split(" "))
df['cleaned_reviews'] = df['cleaned_reviews'].apply(lambda x: remove_sw(x))

In [13]:
df['cleaned_reviews'].head()

0                            [better, get, restaurant]
1    [cut, back, mayo, made, difference, sour, crea...
2    [think, something, wrong, could, taste, cornst...
3    [easily, best, ever, , juicy, flavorful, dry, ...
4                                    [excellent, dish]
Name: cleaned_reviews, dtype: object

In [14]:
import numpy as np 

df['is_positive'] = np.where(df['Rating']<3, 0, 1) 

df[['cleaned_reviews','is_positive']]

,cleaned_reviews,is_positive
0,"[better, get, restaurant]",1
1,"[cut, back, mayo, made, difference, sour, crea...",1
2,"[think, something, wrong, could, taste, cornst...",0
3,"[easily, best, ever, , juicy, flavorful, dry, ...",1
4,"[excellent, dish]",1
...,...,...
1401977,"[disappointed, couldnt, wait, make, husband, b...",0
1401978,"[nothing, drain, dont, heat, liquids, put, mil...",1
1401979,"[good, base, recipe, someone, start, quadruple...",1
1401980,"[thank, much, amazing, recipe, lived, kenai, s...",1


In [15]:
df['cleaned_reviews'] = df['cleaned_reviews'].apply(lambda x: list(filter(None,x)))
        

max_word = max(df.cleaned_reviews, key=len) 
len(max_word)

# import urllib.request 

# url = 'https://nlp.stanford.edu/data/glove.6B.zip'
# filename = 'glove.6b.zip'
# urllib.request.urlretrieve(url,filename)

661

In [16]:
df = df[df.cleaned_reviews.str.len() < 100]

max_word = max(df.cleaned_reviews, key=len) 
len(max_word)

# import zipfile 

# with zipfile.ZipFile('glove.6b.zip', 'r') as zip: 
#     zip.extractall()

99

In [17]:
# word_vectors = dict()

# def add_wordvector(dict, filename):
#     with open(filename, 'r', encoding='utf8') as f:
#         for line in f.readlines():
#             line = line.split(' ')

#             try:
#                 if line[0] not in stop_words:
#                     dict[line[0]] = np.array(line[1:], dtype=float)
#             except:
#                 continue 

def add_wordvector(filename):
    with open(filename, 'r', encoding='utf8') as f:
        wordToGloveVector = {}
        wordToIndex = {}
        indexToWord = {}
        
        
        for line in f.readlines():
            line = line.split(' ')

            try:
                if line[0] not in stop_words:
                    wordToGloveVector[line[0]] = np.array(line[1:], dtype=float)
            except:
                continue
                
        tokens = sorted(wordToGloveVector.keys())
        for i, word in enumerate(tokens):
            kerasIdx = i + 1 # 0 is reserved for masking n Keras 
            wordToIndex[word] = kerasIdx 
            indexToWord[kerasIdx] = word 
            
    return wordToGloveVector, wordToIndex, indexToWord
            
wordToGloveVector, wordToIndex, indexToWord = add_wordvector('glove.6B.50d.txt')
len(stop_words)

179

In [18]:
len(wordToGloveVector)

wordToGloveVector['good'] 

array([-3.5586e-01,  5.2130e-01, -6.1070e-01, -3.0131e-01,  9.4862e-01,
       -3.1539e-01, -5.9831e-01,  1.2188e-01, -3.1943e-02,  5.5695e-01,
       -1.0621e-01,  6.3399e-01, -4.7340e-01, -7.5895e-02,  3.8247e-01,
        8.1569e-02,  8.2214e-01,  2.2220e-01, -8.3764e-03, -7.6620e-01,
       -5.6253e-01,  6.1759e-01,  2.0292e-01, -4.8598e-02,  8.7815e-01,
       -1.6549e+00, -7.7418e-01,  1.5435e-01,  9.4823e-01, -3.9520e-01,
        3.7302e+00,  8.2855e-01, -1.4104e-01,  1.6395e-02,  2.1115e-01,
       -3.6085e-02, -1.5587e-01,  8.6583e-01,  2.6309e-01, -7.1015e-01,
       -3.6770e-02,  1.8282e-03, -1.7704e-01,  2.7032e-01,  1.1026e-01,
        1.4133e-01, -5.7322e-02,  2.7207e-01,  3.1305e-01,  9.2771e-01])

In [19]:
wordToIndex['good']

164285

In [20]:
from nltk.stem import WordNetLemmatizer

# nltk.download('wordnet')
# nltk.download('omw-1.4')

lemmatizer = WordNetLemmatizer()

In [21]:
def lemmatize_token(text):
    lemmatized_token = [lemmatizer.lemmatize(x) for x in text] 
    useful_token = [x for x in lemmatized_token if x in wordToGloveVector]

    return useful_token 

word_token = df['cleaned_reviews'][0] 

lemmatize_token(word_token)


['better', 'get', 'restaurant']

In [22]:
def text_to_vector(text, token_dict=wordToGloveVector):
    processed_token = lemmatize_token(text)

    vectors = []

    for x in processed_token:
        if x not in wordToGloveVector:
            continue 
        
        vectors.append(token_dict[x]) 

    return np.array(vectors, dtype=float) 

def text_to_indices(text):
    processed_token = lemmatize_token(text)
    
    wordToIndices = []
    
    for x in processed_token:
        if x not in wordToGloveVector:
            continue 
        
        wordToIndices.append(wordToIndex[x])
        
        
    return np.array(wordToIndices, dtype=int)


    

In [23]:
# def vectorize_review(df):
#     label = df['is_positive'].to_numpy().astype(int)

#     vectorized_text = []

#     for text in df['cleaned_reviews']:
#         text_as_vector = text_to_vector(text) 

#         if text_as_vector.shape[0] == 0:
#             text_as_vector = np.zeros(shape=(1,50)) 

#         vectorized_text.append(text_as_vector)

#     return vectorized_text, label

def map_input_output(df):
    label = df['is_positive'].to_numpy().astype(int)

    input_text = []

    for text in df['cleaned_reviews']:
#         text_as_indices = text_to_indices(text)
    
#         if text_as_indices.shape[0] == 0:
#             text_as_indices = np.zeros(shape=(1))

        processed_token = lemmatize_token(text)
        
#         if len(processed_token) == False:
#             processed_token = np.zeros(shape=(1)) 

        input_text.append(processed_token)
        
    return input_text, label

In [24]:
train_df = df.sample(frac=.36, random_state=1)
train_df.reset_index(drop=True, inplace=True) 

split_index_1 = int(len(train_df) * 0.7)
split_index_2 = int(len(train_df) * 0.85)

train_df, val_df, test_df = train_df[:split_index_1], train_df[split_index_1:split_index_2], train_df[split_index_2:]

len(train_df), len(val_df), len(test_df)

(350481, 75103, 75104)

In [25]:
x_train, y_train = map_input_output(train_df) 

len(x_train), len(x_train[0]), len(x_train[1])

(350481, 18, 14)

In [26]:
token_len = []

for i in range (len(x_train)):
    token_len.append(len(x_train[i])) 

pd.Series(token_len).describe()

count    350481.000000
mean         25.145871
std          16.206878
min           0.000000
25%          13.000000
50%          22.000000
75%          33.000000
max          99.000000
dtype: float64

In [27]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(x_train)

words_to_index = tokenizer.word_index

# words_to_index

In [28]:
# x_train_indices = tokenizer.texts_to_sequences(x_train)

# x_train_indices

# wordToIndex['recipe']

In [29]:
from tensorflow.keras import layers 
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Embedding

def createPretrainedEmbeddingLayer(wordToGloveVector, wordIndex):
    vocabLen = len(wordIndex) + 1 
    embDim = 50
    
    embeddingMatrix = np.zeros((vocabLen, embDim)) 
    for word, index in wordIndex.items():
        embeddingMatrix[index, :] = wordToGloveVector[word]
        
    embeddingLayer = Embedding(vocabLen, embDim, input_length=100, embeddings_initializer=tf.keras.initializers.Constant(embeddingMatrix), trainable=False)
    
    return embeddingLayer 

In [30]:
gloveEmbeddingLayer = createPretrainedEmbeddingLayer(wordToGloveVector, words_to_index)

wordToGloveVector['easy']

array([-2.3259e-02, -1.0774e-01,  1.9567e-01,  3.3223e-02,  7.4798e-02,
       -2.4640e-01, -3.7363e-01,  2.3737e-01, -4.3820e-02,  7.5206e-01,
       -6.8002e-01,  1.4228e-01, -5.1052e-02,  1.2944e-01,  6.1359e-02,
        1.3980e-01,  6.7984e-01, -5.8867e-01,  1.1884e-01, -1.0120e+00,
       -4.2694e-01,  1.5890e-01, -7.4903e-02,  5.3789e-01,  1.0589e+00,
       -1.3319e+00, -3.8969e-01,  8.3434e-02,  1.2894e+00, -7.9956e-01,
        3.0825e+00,  3.7201e-01, -4.9131e-01,  1.5977e-01,  4.3779e-01,
        1.2637e-01,  1.1982e-01,  9.9909e-01, -2.4347e-01, -7.7499e-01,
        4.7008e-01,  1.7949e-01,  2.6817e-01,  6.6041e-01,  2.0982e-01,
        3.2454e-01,  6.6948e-01, -2.9127e-03,  6.3353e-01,  5.2693e-01])

In [31]:
len(x_train[1133])

0

In [32]:
# x_train = tokenizer.texts_to_sequences(x_train)

# x_train

x_train[0]

['recipe',
 'us',
 '1',
 'cup',
 'cornflakes',
 'instead',
 '2',
 '12',
 'cup',
 'flour',
 'shortening',
 'cut',
 'flour',
 'right',
 'good',
 'lol',
 'thank',
 'posting']

In [42]:
x_train = tokenizer.texts_to_sequences(x_train)

x_train = tf.keras.utils.pad_sequences(x_train, maxlen=100, dtype='float32', padding='post') 

x_train.shape
# tf.keras.utils.pad_sequences([[1, 2, 3], [3, 4, 5, 6], [7, 8]], dtype='float32', padding='post')

AttributeError: 'numpy.ndarray' object has no attribute 'lower'

In [44]:
len(x_train[0]), len(x_train[1]) 

(100, 100)

In [35]:
y_train.shape  


(350481,)

In [36]:
x_val, y_val = map_input_output(val_df)

x_val = tokenizer.texts_to_sequences(x_val)

x_val = tf.keras.utils.pad_sequences(x_val, maxlen=100, dtype='float32', padding='post')

x_val.shape, y_val.shape

((75103, 100), (75103,))

In [37]:
x_test, y_test = map_input_output(test_df)

x_test = tokenizer.texts_to_sequences(x_test)

x_test = tf.keras.utils.pad_sequences(x_test, maxlen=100, dtype='float32', padding='post')

x_test.shape, y_test.shape

((75104, 100), (75104,))

In [43]:
# model = Sequential([])

# # model.add(layers.Input(shape=(100,50))) 
# model.add(gloveEmbeddingLayer)
# model.add(layers.LSTM(460, return_sequences=True))
# model.add(layers.Dropout(0.2))
# model.add(layers.Flatten())
# model.add(layers.Dense(128, activation='relu'))
# model.add(layers.Dense(1, activation='sigmoid'))

# model.summary()

In [44]:
modelGRU = Sequential([])

# modelGRU.add(layers.Input(shape=(100,50)))
modelGRU.add(gloveEmbeddingLayer)
modelGRU.add(layers.GRU(46, return_sequences=True))
modelGRU.add(layers.Dropout(0.2))
modelGRU.add(layers.Flatten())
modelGRU.add(layers.Dense(1, activation='sigmoid')) 

modelGRU.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 50)           1654500   
                                                                 
 gru (GRU)                   (None, 100, 46)           13524     
                                                                 
 dropout (Dropout)           (None, 100, 46)           0         
                                                                 
 flatten (Flatten)           (None, 4600)              0         
                                                                 
 dense (Dense)               (None, 1)                 4601      
                                                                 
Total params: 1,672,625
Trainable params: 18,125
Non-trainable params: 1,654,500
_________________________________________________________________


2022-06-30 19:28:21.525773: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-30 19:28:21.526708: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-30 19:28:21.526750: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-06-30 19:28:21.526786: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-06-30 19:28:21.528756: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Co

In [45]:
# modelRNN = Sequential([])

# # modelRNN.add(layers.Input(shape=(100,50)))
# modelRNN.add(gloveEmbeddingLayer)
# modelRNN.add(layers.SimpleRNN(68, return_sequences=True))
# modelRNN.add(layers.Dropout(0.2))
# # modelRNN.add(layers.SimpleRNN(68)) 
# modelRNN.add(layers.Flatten())
# modelRNN.add(layers.Dense(1, activation='sigmoid')) 

# modelRNN.summary()

In [46]:
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam 
from tensorflow.keras.metrics import AUC 
from tensorflow.keras.callbacks import ModelCheckpoint 

# model_cp = ModelCheckpoint('model_embed_reludense/', save_best_only=True)
# model.compile(optimizer=Adam(learning_rate=0.001), loss=BinaryCrossentropy(), metrics=['accuracy', AUC(name='auc')]) 

# modelrnn_cp = ModelCheckpoint('modelrnn_embed/', save_best_only=True)
# modelRNN.compile(optimizer=Adam(learning_rate=0.001), loss=BinaryCrossentropy(), metrics=['accuracy', AUC(name='auc')])

modelgru_cp = ModelCheckpoint('modelgru_embed2/', save_best_only=True)
modelGRU.compile(optimizer=Adam(learning_rate=0.001), loss=BinaryCrossentropy(), metrics=['accuracy', AUC(name='auc')])


In [47]:
class_freq = pd.value_counts(train_df['is_positive']) 
class_freq

1    322925
0     27556
Name: is_positive, dtype: int64

In [48]:
weights = {0: class_freq.sum() / (class_freq[0] * 2), 1: class_freq.sum() / (class_freq[1] * 2)}   #model1

# weights = {0: class_freq.sum() / class_freq[0], 1: class_freq.sum() / class_freq[1] } #model2

# weights = {0: (class_freq.sum() / (class_freq[0]))+3, 1: class_freq.sum() / (class_freq[1])} #model3

weights 

{0: 6.359431702714472, 1: 0.5426662537740962}

In [49]:
# model.fit(x_train, y_train, validation_data=(x_val,y_val), epochs=15, callbacks=[model_cp], class_weight=weights) 

# modelRNN.fit(x_train, y_train, validation_data=(x_val,y_val), epochs=15, callbacks=[modelrnn_cp], class_weight=weights) 

modelGRU.fit(x_train, y_train, validation_data=(x_val,y_val), epochs=15, callbacks=[modelgru_cp], class_weight=weights) 

# modelGRU.fit(x_train, y_train, validation_data=(x_val,y_val), epochs=15, class_weight=weights) 

Epoch 1/15
10952/10953 [============================>.] - ETA: 0s - loss: 0.5603 - accuracy: 0.7209 - auc: 0.7841

INFO:tensorflow:Assets written to: modelgru_embed2/assets


INFO:tensorflow:Assets written to: modelgru_embed2/assets


10953/10953 [==============================] - 346s 32ms/step - loss: 0.5603 - accuracy: 0.7209 - auc: 0.7841 - val_loss: 0.6228 - val_accuracy: 0.6594 - val_auc: 0.8129
Epoch 2/15
10952/10953 [============================>.] - ETA: 0s - loss: 0.5236 - accuracy: 0.7623 - auc: 0.8150

INFO:tensorflow:Assets written to: modelgru_embed2/assets


INFO:tensorflow:Assets written to: modelgru_embed2/assets


10953/10953 [==============================] - 342s 31ms/step - loss: 0.5236 - accuracy: 0.7623 - auc: 0.8150 - val_loss: 0.4106 - val_accuracy: 0.8378 - val_auc: 0.8236
Epoch 3/15
10953/10953 [==============================] - 339s 31ms/step - loss: 0.5102 - accuracy: 0.7758 - auc: 0.8253 - val_loss: 0.4303 - val_accuracy: 0.8283 - val_auc: 0.8276
Epoch 4/15
10953/10953 [==============================] - 339s 31ms/step - loss: 0.5010 - accuracy: 0.7837 - auc: 0.8323 - val_loss: 0.4863 - val_accuracy: 0.7864 - val_auc: 0.8291
Epoch 5/15
10953/10953 [==============================] - 342s 31ms/step - loss: 0.4916 - accuracy: 0.7903 - auc: 0.8396 - val_loss: 0.4702 - val_accuracy: 0.7969 - val_auc: 0.8322
Epoch 6/15
10953/10953 [==============================] - 343s 31ms/step - loss: 0.4859 - accuracy: 0.7933 - auc: 0.8438 - val_loss: 0.5571 - val_accuracy: 0.7446 - val_auc: 0.8328
Epoch 7/15
10953/10953 [==============================] - 341s 31ms/step - loss: 0.4798 - accuracy: 0.7961

In [50]:
# modelGRU.save('gru_embed.h5')

In [38]:
from tensorflow.keras.models import load_model 

best_model = load_model('modelgru_embed2/') 

# best_modelrnn = load_model('modelrnn/')

# best_modelgru = load_model('modelgru/')


2022-07-01 09:56:03.663412: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-01 09:56:03.664577: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-01 09:56:03.664630: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-07-01 09:56:03.664684: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-07-01 09:56:03.666571: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Co

In [39]:
test_predictions = (best_model.predict(x_test) > 0.5)

from sklearn.metrics import classification_report

print(classification_report(y_test, test_predictions))

2347/2347 [==============================] - 17s 7ms/step
              precision    recall  f1-score   support

           0       0.26      0.62      0.37      5871
           1       0.96      0.85      0.91     69233

    accuracy                           0.84     75104
   macro avg       0.61      0.73      0.64     75104
weighted avg       0.91      0.84      0.86     75104



In [40]:
best_model.evaluate(x_test, y_test)

2347/2347 [==============================] - 17s 7ms/step - loss: 0.4140 - accuracy: 0.8358 - auc: 0.8171


[0.41402214765548706, 0.8358409404754639, 0.8171298503875732]

In [41]:
best_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 50)           1654500   
                                                                 
 gru (GRU)                   (None, 100, 46)           13524     
                                                                 
 dropout (Dropout)           (None, 100, 46)           0         
                                                                 
 flatten (Flatten)           (None, 4600)              0         
                                                                 
 dense (Dense)               (None, 1)                 4601      
                                                                 
Total params: 1,672,625
Trainable params: 18,125
Non-trainable params: 1,654,500
_________________________________________________________________


model2 = load_model('model2/')

In [53]:
model2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 100, 460)          940240    
                                                                 
 dropout_3 (Dropout)         (None, 100, 460)          0         
                                                                 
 flatten_1 (Flatten)         (None, 46000)             0         
                                                                 
 dense_1 (Dense)             (None, 1)                 46001     
                                                                 
Total params: 986,241
Trainable params: 986,241
Non-trainable params: 0
_________________________________________________________________


In [51]:
model_embed = load_model('model_embed/')

In [52]:
model_embed.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 50)           19992600  
                                                                 
 lstm (LSTM)                 (None, 100, 460)          940240    
                                                                 
 dropout (Dropout)           (None, 100, 460)          0         
                                                                 
 flatten (Flatten)           (None, 46000)             0         
                                                                 
 dense (Dense)               (None, 1)                 46001     
                                                                 
Total params: 20,978,841
Trainable params: 986,241
Non-trainable params: 19,992,600
_________________________________________________________________
